### Program to scrape names of all movies and their corresponding info by year released:
### (calendar gross, max theaters, total gross, release date, distribution studio, and url)

In [57]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs4

In [102]:
# Loop over years:
year_range = [*range(1977,2020)]

for yr in range(year_range[0],year_range[-1]+1):
    year = str(yr)
    url  =('https://www.boxofficemojo.com/year/'+year+'/?grossesOption=calendarGrosses')
#url  =('https://www.boxofficemojo.com/year/2019/?grossesOption=calendarGrosses')
    html = requests.get(url)
    soup = bs4(html.text, 'lxml')
    
    table_data = [x.text for x in soup.select('tr td')[0:15000]]

    # removes 'false' list items: *NOTE: This does not work for years 2002 and 2018*
    # 2002 and 2018 contain 'true' entries that need to be removed too
    temp = list(filter(lambda x: x!= 'false',table_data))
    clean_table = list(filter(lambda x: x!= 'true',temp))

    nmax  = len(clean_table)
    ncols = 10

# Create dataframe columns from annual movie data:
    df = pd.DataFrame()
    df[0]=clean_table[0:nmax:ncols]
    df[1]=clean_table[1:nmax:ncols]
    df[2]=clean_table[5:nmax:ncols]
    df[3]=clean_table[6:nmax:ncols]
    df[4]=clean_table[7:nmax:ncols]
    df[5]=clean_table[8:nmax:ncols]
    df[6]=clean_table[9:nmax:ncols]
        
    df.columns=[str(year_range[0]) + ' Rank',
                'Release',
                'Calendar Gross',
                'Max Theaters',
                'Total Gross',
                'Release Date',
                'Distributor']
    
    #=================================
    # Grab the URL of each movie in the list:
    newlink=[]
    for a in soup.find_all('a', href=True):
        if a['href'].startswith('/release/rl'):
            l = a['href'].split('?')
            l = 'boxofficemojo.com'+l[0]
            newlink.append(l)
    
    df['URL'] = newlink
    
    # Write dataframe to .csv file:
    df.to_csv(year+'_domestic_releases.csv', index = False)

    #=================================
    #wait 3 seconds between iterations
    time.sleep(3)